**[필요한 라이브러리 호출 및 API키 설정]**

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")    # https://dashboard.cohere.com/ (필요하시면 말씀하세요)

In [2]:
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables.history import BaseChatMessageHistory, RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

c:\Users\user\Desktop\MODULABS\LangchainThon\LangchainThon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **[데이터 임베딩]**

**[청크 분할]**

In [3]:
# Chunk split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]             
)

**[PDF 문서 로드/분할 및 벡터 임베딩]**

In [4]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\pdf\모두연 브랜딩북 정리.pdf"

loader_pdf = PyPDFLoader(PDF_PATH)
pages_pdf = loader_pdf.load()

for d in pages_pdf:
    d.metadata["source_type"] = "pdf"
    d.metadata["source"] = os.path.basename(PDF_PATH)

docs_pdf = text_splitter.split_documents(pages_pdf)

**[HTML 문서 로드/분할 및 벡터 임베딩]**

In [5]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

HTML_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\LMS oops 해결법.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\LMS 아이펠 노트북이 아닙니다 에러.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\LMS 이용시 발생하는 문제 해결법.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\교육과정 중 취업 시.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\데싸 5기 훈련 정보.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\수강 중 고용 형태 관련 안내.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\스터디를 만들고 싶은데 어떻게 해야 하나요.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\오프닝 장소와 클로징 장소가 다릅니다.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\인터넷이 불안정하여 출결 QR을 제대로 찍지 못하였습니다.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\제적 가이드.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\출결 및 공가에 대하여.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\툴 세팅.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\훈련 장려금 지급 확인.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\훈련 참여 규칙.html",
]

html_list = []

# 각 파일 로드 + Metadata 저장
for path in HTML_PATH:
    loader_html = UnstructuredHTMLLoader(path)
    pages_html = loader_html.load()

    for d in pages_html:
        d.metadata["source_type"] = "html"
        d.metadata["source"] = os.path.basename(path)

    html_list.extend(pages_html)  

docs_html = text_splitter.split_documents(html_list)

**[WORD 문서 로드/분할 및 벡터 임베딩]**

In [6]:
from langchain_community.document_loaders import Docx2txtLoader

WORD_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\word\휴가신청서(데싸_5기).docx"
loader_word = Docx2txtLoader(WORD_PATH)
pages_word = loader_word.load()

for d in pages_word:
    d.metadata["source_type"] = "word"
    d.metadata["source"] = os.path.basename(WORD_PATH)

docs_word = text_splitter.split_documents(pages_word)

**[CSV 문서 로드/분할 및 벡터 임베딩]**

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

CSV_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 일정표.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 동료들.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 운영진.csv"
]

csv_list = []

# 각 파일 로드
for path in CSV_PATH:
    loader_csv = CSVLoader(path, encoding = 'cp949')
    pages_csv = loader_csv.load()

    for d in pages_csv:
        d.metadata["source_type"] = "csv"
        d.metadata["source"] = os.path.basename(path)

    csv_list.extend(pages_csv)  

docs_csv = text_splitter.split_documents(csv_list)

**[벡터db 저장]**

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

# ChromaDB 벡터 임베딩 후 저장
vectorstore = Chroma.from_documents(docs_html, OpenAIEmbeddings(model='text-embedding-3-small'))

vectorstore.add_documents(docs_word)
vectorstore.add_documents(docs_csv)
vectorstore.add_documents(docs_pdf)

['e4f8decd-0f0c-4473-882c-de7f464dc400',
 '81826633-f3a2-47c8-9314-5a8626c097e9',
 '09ff9266-2a2b-49f3-bb2c-7db14ce97331',
 'eb4eff81-d6b0-460c-a2f6-fa91217c92e6',
 '172d0892-1b9f-406f-ad5a-021aebd35077',
 '40ad8661-aa64-4fea-9ed3-f0ed12c0627d',
 '51c128db-4991-49e1-9c7f-bbca6ba6c3b8',
 '0b553829-22ef-4be9-8ae4-5a530cd2016c']

## **[RAG 시스템]**

**[HyDE]**

In [9]:
# prompt 

from langchain.prompts import PromptTemplate
from langchain_core.retrievers import BaseRetriever
from typing import List, Optional
from langchain_core.documents import Document

hyde_template = """아래 질문에 대한 배경 설명을 짧은 단락 1~2개의 한국어로 작성하세요.
핵심 키워드(이름, 상태, 날짜 등)를 분명히 포함하세요. 사실을 꾸미지 말고 중립적으로 기술만 하세요.

질문: {question}

[가상의 배경 문서]"""
hyde_prompt = PromptTemplate.from_template(hyde_template)

In [10]:
# retriever

class HyDERetriever(BaseRetriever):
    # ▼ pydantic 필드 선언(필수)
    vectorstore: Chroma
    llm: ChatOpenAI
    prompt: PromptTemplate
    k: int = 16
    fetch_k: int = 64
    lambda_mult: float = 0.4
    use_mmr: bool = True

    # pydantic v1 호환 설정(임의 타입 허용)
    class Config:
        arbitrary_types_allowed = True

    # pydantic v2를 쓰는 환경이면:
    # model_config = {"arbitrary_types_allowed": True}

    def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        # 1) 질문 → 가상 문서(HyDE)
        hyde_text = self.llm.invoke(self.prompt.format(question=query)).content
        # 2) 가상 문서 임베딩 → by_vector 검색
        q_vec = self.vectorstore._embedding_function.embed_query(hyde_text)
        if self.use_mmr:
            docs = self.vectorstore.max_marginal_relevance_search_by_vector(
                embedding=q_vec, k=self.k, fetch_k=self.fetch_k, lambda_mult=self.lambda_mult
            )
        else:
            docs = self.vectorstore.similarity_search_by_vector(q_vec, k=self.k)
        return docs

    # (선택) async 버전도 구현 가능
    async def _aget_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        hyde_text = (await self.llm.ainvoke(self.prompt.format(question=query))).content
        q_vec = self.vectorstore._embedding_function.embed_query(hyde_text)
        if self.use_mmr:
            docs = self.vectorstore.max_marginal_relevance_search_by_vector(
                embedding=q_vec, k=self.k, fetch_k=self.fetch_k, lambda_mult=self.lambda_mult
            )
        else:
            docs = self.vectorstore.similarity_search_by_vector(q_vec, k=self.k)
        return docs
    
# HyDE 기반 1차 후보 검색기
hyde_retriever = HyDERetriever(
    vectorstore=vectorstore,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    prompt=hyde_prompt,
    k=32,             # 후보 폭 (rerank 전에 넉넉히)
    fetch_k=96,
    lambda_mult=0.4,
    use_mmr=True,
)

C:\Users\user\AppData\Local\Temp\ipykernel_22760\3093843678.py:3: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class HyDERetriever(BaseRetriever):


In [11]:
# rerank, reorder

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_cohere import CohereRerank

reranker = CohereRerank(
    cohere_api_key=os.getenv("COHERE_API_KEY"),
    model="rerank-multilingual-v3.0",
    top_n=10,  # 최종 컨텍스트 개수
)
reorder = LongContextReorder()
compressor = DocumentCompressorPipeline(transformers=[reranker, reorder])

# 최종 리트리버 = HyDE(후보) → Cohere Rerank(선별) → Reorder(순서)
upgraded_retriever = ContextualCompressionRetriever(
    base_retriever=hyde_retriever,
    base_compressor=compressor,
)

**[채팅 히스토리와 사용자 질문 통합]**

In [12]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.retrievers.document_compressors import LLMChainExtractor
from datetime import datetime, timezone
from zoneinfo import ZoneInfo


# =========== Cohere API KEY 필요 ============

# # Reranking 이전 base 
# base_retriever = vectorstore.as_retriever(
#     search_type="mmr", 
#     search_kwargs={"lambda_mult": 0.4, "fetch_k": 96, "k": 48}
# )

# # Rerank
# reranker = CohereRerank(
#     model="rerank-multilingual-v3.0",    
#     top_n=10                              
# )

# # Reorder
# reorder = LongContextReorder()

# # Rerank + Reorder
# compressor = DocumentCompressorPipeline(transformers=[reranker, reorder])

# upgraded_retriever = ContextualCompressionRetriever(
#     base_retriever=base_retriever,
#     base_compressor=compressor            
# )
# ===========================================




# ======== Cohere API KEY 없는 경우 ==========

# retriever = vectorstore.as_retriever(
#     search_type="mmr",                      
#     search_kwargs={"lambda_mult": 0.3, "fetch_k": 32, "k": 8, }  
# )

# # Long-Context Reorder 활용하여 컨텍스트 재정렬
# reordering = LongContextReorder()
# compressor = DocumentCompressorPipeline(transformers=[reordering])

# reordered_retriever = ContextualCompressionRetriever(
#     base_retriever=retriever,
#     base_compressor=compressor
# )
# ===========================================


# LLM 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 현재 시간 선언
KST = ZoneInfo("Asia/Seoul")
today_str = datetime.now(KST).strftime("%Y-%m-%d %H:%M:%S")

# 질문 프롬프트
contextualize_q_system_prompt = """

이전 대화가 있다면 참고하여,
사용자의 최신 질문을 독립적으로 이해 가능한 한 문장으로 바꿔주세요.
답변하지 말고 질문만 재작성하세요.

"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


history_aware_retriever = create_history_aware_retriever(
    llm,
    upgraded_retriever,             # Cohere API KEY 없는 경우 >> reordered_retriever
    contextualize_q_prompt
)


# 답변 프롬프트
qa_system_prompt = """

당신은 '모두의연구소(모두연)' 수강생들의 비서입니다.
현재 시간은 {today} (KST)입니다. 사용자의 '어제, 내일' 등의 표현은 {today}를 기준으로 파악하세요.
제공된 문서 내용만을 근거로 답하세요. 근거가 없으면 '정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.'라고만 대답하세요.
사용자 입력에 포함된 사실은 근거로 사용하지 마세요.
최대 3문장으로 짧게 답변하세요.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

**[RAG 체인 구축]**

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

**[세션별 기록 저장]**

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#채팅 세션별 기록 저장 위한 Dictionary 선언
store = {}

#주어진 session_id 값에 매칭되는 채팅 히스토리 가져오는 함수 선언
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


#RunnableWithMessageHistory 모듈로 rag_chain에 채팅 기록 세션별로 자동 저장 기능 추가
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## **[실험 결과]**

### **[RUN 1]**

- 성능: 테스트 6,7,8 정답 / 나머지 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.5
        - fetch_k: 24
        - k: 6

### **[RUN 2]**

- 성능: 테스트 1, 6, 7, 8 정답 / 2, 3 부분 정답(오답 섞임) / 4, 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.3
        - fetch_k: 32
        - k: 8
    - LongContext Reorder
        - k: 32
        - top_n: 8
        - model: rerank-multilingual-v3.0

### **[RUN 3]**

- 성능: 테스트 1, 4, 6, 7, 8 정답 / 2, 3 부분 정답(오답 섞임) / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever
    - type: similarity
    - kwargs
        - k: 32
    - cohere rerank
        - top_n: 8
        - model: rerank-multilingual-v3.0

### **[RUN 4]**

- 성능: 테스트 1, 2, 3, 4, 6, 7, 8 정답 / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.3
        - fetch_k: 64
        - k: 32
    - cohere rerank
        - k: 32
        - top_n: 8
        - model: rerank-multilingual-v3.0
    - LongContext Reorder

### **[RUN 5]**

- 성능: 테스트 1, 2, 3, 4, 6, 7, 8 정답 / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.4
        - fetch_k: 96
        - k: 48
    - cohere rerank
        - top_n: 10
        - model: rerank-multilingual-v3.0
    - LongContext Reorder  

- prompt:  
    - 답변 프롬프트 추가  
        - 당신은 '모두의연구소(모두연)' 수강생들의 비서입니다.
        - 사용자 입력에 포함된 사실은 근거로 사용하지 마세요.  
        - 현재 시간은 {today} (KST)입니다. '어제, 내일' 등의 표현은 {today}를 기준으로 파악하세요.  

- 기타
    - 현재 시간 메서드 추가  

### **[RUN 6]**

- HyDE
- rerank, reorder


- MultiRetrievalQAChain 활용?
    - 인덱싱에 대해 description을 자세히 작성하고, LLM에게 쿼리에 맞는 적절한 옵션(파일)을 고르게끔 함  
    - ex. 
        - 출결 총 횟수 >> '데싸 ~.csv 확인'

- text splitter 파라미터 실험  

- retriever 파라미터 실험  

In [15]:
# ===================
# 테스트 1
# ===================

#채팅 히스토리를 적재하기 위한 리스트
chat_history = []

question = "몇 시부터 지각이야?"

#첫 질문에 답변하기 위한 rag_chain 실행
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

#첫 질문과 답변을 채팅 히스토리로 저장
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "조퇴는?"

#두번째 질문 입력 시에는 첫번째 질문-답변이 저장된 chat_history가 삽입됨
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

조퇴는 14시 30분부터 17시 49분 퇴실하는 경우를 말합니다.


In [16]:
# ===================
# 테스트 2
# ===================

chat_history = []

question = "몇 시부터 지각이야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "휴가 신청서 작성해야 한다고 들었는데, 어떤 서류야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

휴가 신청서는 '휴가신청서(데싸 5기).docx' 양식을 사용하여 작성해야 합니다. 신청서는 휴가 사용 전 제출해야 하며, 당일 또는 발생일 이후에는 사용이 불가합니다.


In [17]:
# ===================
# 테스트 3
# ===================

question = "휴가쓰고 싶어. 휴가 신청하려면 어떻게 해야해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

휴가를 사용하기 위해서는 휴가신청서를 작성하여 제출해야 합니다. 신청서는 발생일 이전에 제출해야 하며, 당일 또는 발생일 이후에는 사용이 불가합니다. 자세한 내용은 휴가신청서(데싸 5기).docx를 참고하세요.


In [37]:
# ===================
# 테스트 4
# ===================

chat_history = []

question = "데이터 사이언스 5기 동료들 중 MBTI가 ISTJ 인 사람은 누구야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "그 사람들의 취미는 뭐야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.


In [19]:
# ===================
# 테스트 4 - BONUS
# ===================

question = "데이터 사이언스 5기 동료들 중 MBTI가 ISTJ 인 사람은 누구야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.


In [20]:
# ===================
# 테스트 5
# ===================

question = "나는 데싸5기 손호진이야. 오늘은 2025년 10월 26일인데, 지금까지 내가 총 조퇴와 결석을 몇 번 했는지 궁금해."
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.


In [21]:
# ===================
# 테스트 6
# ===================

question = "모두연이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

모두연은 자유롭게 연구하고 배우는 커뮤니티로, 경쟁 없이 집단 지성을 통해 학습하는 곳입니다. 상생과 성장을 추구하며, 스스로 질문하고 답을 찾아가는 환경을 제공합니다. 기존의 교육 방식을 바꾸려는 목표를 가지고 있습니다.


In [22]:
# ===================
# 테스트 7
# ===================

question = "모두의 연구소의 핵심 슬로건이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

모두의 연구소의 핵심 슬로건은 ‘쉐벨그투’로, 이는 '쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)'의 약자입니다.


In [23]:
# ===================
# 테스트 8
# ===================

question = "쉐벨그투에 대해 알려줘."
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

'쉐벨그투'는 '쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)'의 약자로, 모두연의 핵심 슬로건입니다. 이 슬로건은 함께 가치를 나누고 성장하자는 의미를 담고 있습니다.


In [24]:
# ===================
# 테스트 9
# ===================

chat_history = []

question = "예비군 훈련으로 출석을 못할거 같은데 어떻게 해야해?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "제출해야하는 서류있어?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

third_question = "병원 가고 싶을 때는?"

ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_3["answer"])

병원 진료로 인한 결석은 공결로 인정됩니다. 진료 확인서, 통원 확인서, 처방전, 진단서, 입/퇴원 확인서 중 하나를 제출해야 하며, 본인 및 자녀(만 19세 미만)의 진료에만 해당됩니다.


In [25]:
# ===================
# 테스트 10
# ===================

question = "훈련 장려금에 대해 알려줘."
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

훈련 장려금은 단위 기간 내 80% 이상 출석 시 지급됩니다. 지급 금액은 1일 15,800원으로, 출석 일수에 따라 최대 20일 기준으로 계산됩니다. 고용 형태에 따라 지급되며, 고용 형태가 변경될 경우 반드시 알려야 합니다.


In [26]:
# ===================
# 테스트 11
# ===================

question = "스터디 참여하고 싶은데 어떻게 해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

스터디에 참여하고 싶으시면, 디스코드에서 자율적으로 모집된 스터디에 참여하시면 됩니다. 어떤 스터디에 참여할지 담당 퍼실에게 공유하시면 노션 페이지와 전용 채널 생성에 도움을 받을 수 있습니다.


In [27]:
# ===================
# 테스트 12
# ===================

question = "훈련 기간 중 회사 면접이 생겼어. 어떻게 해야해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

부득이한 사정으로 출석이 어려운 경우, 운영진에게 사전에 알려야 합니다. 면접 일정에 맞춰 외출 처리도 잊지 마세요. 자세한 사항은 운영매니저님께 문의해주세요.


In [28]:
# ===================
# 테스트 13
# ===================

question = "오전에 인터넷 문제로 QR을 못 찍었어. 출석 인정 가능해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

정상적으로 출석을 하셨다는 기록이 존재해야 전산 처리가 가능하므로, 입실 QR을 찍지 못하셨다면 출석 인정이 어려울 수 있습니다. 하지만, 입실 때 화면 캡쳐를 통해 증명할 수 있다면 도움을 받을 수 있습니다. 운영진에게 문의해 보시기 바랍니다.


In [29]:
# ===================
# 테스트 14
# ===================

question = "집에서 말고 다른 곳에서 수업 들어도돼?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

사전에 승인되지 않은 외부 장소(카페 등)에서 수업을 듣는 것은 무단 외출로 간주될 수 있습니다. 따라서 집에서 수업을 듣는 것이 권장됩니다. 추가적인 문의는 운영매니저님이나 퍼실님께 하시기 바랍니다.


In [30]:
# ===================
# 테스트 15
# ===================

question = "수료하려면 어떻게 해야해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

수료를 위해서는 교육 과정에 성실하게 참여하고, 결석하지 않도록 유의해야 합니다. 또한, 적극적으로 수업에 참여하고 다른 사람의 이야기에 반응해야 합니다. 자세한 사항은 운영매니저님이나 퍼실님께 문의해주세요.


In [31]:
# ===================
# 테스트 16
# ===================

chat_history = []

question = "규칙이 있어?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "구체적으로 알려줘"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

참여 규칙은 다음과 같습니다:

1. 교육에 성실하게 참여하고, 부득이한 상황을 제외하고 결석하지 않도록 유의합니다.
2. 적극적으로 수업에 참여하고, 다른 사람의 이야기에 반응합니다.
3. 상대방이 불쾌감을 느낄 수 있는 말이나 행동은 삼가며, 기본적인 예의를 갖추고 친절하게 대합니다.

더 자세한 내용은 문서를 참고해 주세요.


In [32]:
# ===================
# 테스트 17
# ===================

question = "LMS에 Oooooo00ops 라는 에러가 뜨는데 이거 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

LMS에서 "Oooooo00ops" 에러가 발생하는 경우는 여러 가지가 있을 수 있습니다. 일반적인 해결 방안으로는 새로고침(F5), 강력 새로고침(Shift + F5), 로그아웃 후 재로그인, 브라우저 종료 후 재접속, 다른 노드 접속 후 컨테이너 초기화 등이 있습니다. 만약 이러한 방법으로도 해결되지 않는다면 퍼실에게 문의해 주세요.


In [33]:
# ===================
# 테스트 18
# ===================

question = "아이펠 노트북이 아니라는데?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

이 에러는 다른 기기에서 LMS에 접속할 때 발생할 수 있습니다. 인터넷 사용기록(쿠키/캐시)을 삭제하거나, 이전에 접속했던 기기에서 로그인해 보세요. 그래도 문제가 해결되지 않으면 퍼실이나 운영매니저에게 문의해주세요.


In [34]:
# ===================
# 테스트 19
# ===================

question = "내일까지만 수업에 참여할거같아. 수료 가능할까?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.


In [35]:
# ===================
# 테스트 20
# ===================

question = "난 여행을 좋아하는데, 나랑 같은 취미인 사람 있으려나?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

정보가 명확하지 않습니다. 운영매니저님이나 퍼실님께 문의해주세요.


주로 출결, 훈련 장려금 내용=> 만약, 봇이 있다면, ' 나는 단위기간 29일(30일) 중에 며칠 결석했는데 얼마 받아요?' 알려주면 좋을 듯!
>> 어디서 정보 확인 가능?

과정 80% 수료 조건 관련 질문
데싸 5기 기준 120일 중 96일(25년 12월 3일 기준)이후 과정 마무리시 80%인정
(그전 drop시 '수강', 96일 이후 '수료', 전일완료 '졸업')
취업 후, 수료 이상이면 이력서상 경력 인정도 되고 유리하지만, 그 이전 drop시는 추후 재직자 훈련 과정 등등 5년 이내 수강 불가능해 불이익
=> '저는 언제부터 수료 인정되나요?' 등

공결의 경우, 제출서류 확인 요청 (진료확인서, 진단서, 처방전, 예비군 훈련 필증, 면접확인서 등)
=> ' 어떤 서류 제출해야 되요?' 바로 안내되면 좋을 듯!